# DATA PREPROCESSING

In [1]:
import os
import numpy as np
import pandas as pd
from scipy.sparse import *
from scipy import *
from sklearn.preprocessing import LabelEncoder

In [2]:
# https://grouplens.org/datasets/book-crossing/
# load book_crossing dataset 1 million ratings , ~270k users and ~270k bookids, did preprocessing with few columns and rows
data = pd.read_excel('Book_Ratings.xlsx') # encoding='ISO-8859-1'


In [3]:
nRow, nCol = data.shape
print(f'There are {nRow} rows and {nCol} columns')
data.head()

There are 1048574 rows and 3 columns


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0.0
1,276726,155061224,5.0
2,276727,446520802,0.0
3,276729,052165615X,3.0
4,276729,521795028,6.0


In [3]:
data.dtypes

User-ID          int64
ISBN            object
Book-Rating    float64
dtype: object

In [4]:
sample_data = data.rename(columns={'User-ID': 'userid', 'ISBN': 'movieid', 'Book-Rating': 'movieRating'})
sample_data

,userid,movieid,movieRating
0,276725,034545104X,0.0
1,276726,155061224,5.0
2,276727,446520802,0.0
3,276729,052165615X,3.0
4,276729,521795028,6.0
...,...,...,...
1048569,250764,451410777,0.0
1048570,250764,452264464,8.0
1048571,250764,048623715X,0.0
1048572,250764,486256588,0.0


In [68]:
# data cleaning
# df = sample_data
# df['movieid'] = df['movieid'].astype(str)
# print(df.dtypes)
# df['movieid'] = df['movieid'].str.replace('\D+','')
# # df['movieid'] = df['movieid'].str.replace('[^0-9]', '')

# def numbers(element):
#   if type(element) == str: 
#     return "".join(filter(str.isnumeric, element))

# df.loc[:,'num'] = [numbers(x) for x in df.movieid]
# # df['num'].astype(float)
# print(df.head(10))
# print(df.dtypes)
# df[df['movieid'] == ''].index
# # pd.isnull('movieid')
# # df.isnull().sum().sum()
# # df.dropna()
# # df
# df.iloc[531937]

userid           int64
movieid         object
movieRating    float64
num             object
dtype: object
   userid     movieid  movieRating         num
0  276725   034545104          0.0   034545104
1  276726   155061224          5.0   155061224
2  276727   446520802          0.0   446520802
3  276729   052165615          3.0   052165615
4  276729   521795028          6.0   521795028
5  276733  2080674722          0.0  2080674722
6  276736  3257224281          8.0  3257224281
7  276737   600570967          6.0   600570967
8  276744   038550120          7.0   038550120
9  276745   342310538         10.0   342310538
userid           int64
movieid         object
movieRating    float64
num             object
dtype: object


userid            128375
movieid        068484477
movieRating            0
num            068484477
Name: 531937, dtype: object

In [5]:
frequency = sample_data.groupby('userid').count()
print(len(frequency))
test_users = []
for i in range(len(frequency)):
    fre = frequency.iloc[i]
    break
    if (fre[0] > 100):
        test_users.append(i+1)
#print(len(test_users))

95512


In [6]:
test_userid = []
test_movieid = []
test_movieRating = []

In [7]:
print(sample_data.userid.max())


278854


create testing set

In [8]:
# if the user has rated more than "50" movies then add him to test dataset
for user in range(1,278855):
    filter_df = sample_data.query("userid == " + str(user))
    if (len(filter_df) < 50):
        continue
    # add 20% of that particular user data to the testdata set 
    portion = int(len(filter_df)*(0.2))
    random_indices = set(np.random.choice(len(filter_df), portion))
    for i in random_indices:
        row = filter_df.iloc[i]
        test_userid.append(row.userid)
        test_movieid.append(row.movieid)
        test_movieRating.append(row.movieRating)
 

In [9]:
print(len(test_userid))
print(len(test_movieid))
print(len(test_movieRating))

126624
126624
126624


In [10]:
test_df_dict = {"userid": test_userid, "movieid":test_movieid, "movieRating":test_movieRating}
convert_dict = {
    "userid" :      int,
    "movieid" :     object,
    "movieRating"  :   int
    }
test_df = pd.DataFrame(test_df_dict)
test_df = test_df.astype(convert_dict)
print(test_df.head())
test_df.shape

   userid     movieid  movieRating
0     183  9728487606            6
1     183   100940/86            9
2     183    10745/85            0
3     183   350583378            0
4     183  40827/3218            0


(126624, 3)

In [13]:
test_df.to_csv("test_ratings.csv",index=False)
# test dataset csv

In [14]:
test_data = pd.read_csv('test_ratings.csv')
test_users = test_data.userid.unique()
print(len(test_users))
test_users_list = test_users.tolist()
with open ('test_users.txt', 'w') as fo:
     fo.write(','.join(str(i) for i in test_users_list))

3115


In [15]:
# data redundancy
# adding unique userid,movieid tuple to the test_tuple_set
#  trying to avoid users who have rated same movie twice
test_tuple_set = set()
print(len(test_userid))
for i in range(len(test_userid)):
    userid_movieid_tuple = (test_userid[i], test_movieid[i])
    if userid_movieid_tuple in test_tuple_set:
        print(userid_movieid_tuple)
        print(":)")
    test_tuple_set.add(userid_movieid_tuple)
print(len(test_tuple_set))
print(len(test_df))

126624
(120548, 0)
:)
126623
126624


create Training set

In [16]:
def drop_rating(row):
    print(len(row))
    t = (row.userid, row.movieid)
    if t in test_tuple_set:
        row.movieRating = 0.0
    return row

In [17]:
# removing test data values/ratings from the training dataset and making them zero not deleting 
for i in range(len(sample_data)): # len(sample_data)
    row = sample_data.iloc[i]
    t = (row.userid, row.movieid)
    if t in test_tuple_set:
        sample_data.at[i,'movieRating'] = 0.0


In [18]:
sample_data.head(10)

,userid,movieid,movieRating
0,276725,034545104X,0.0
1,276726,155061224,5.0
2,276727,446520802,0.0
3,276729,052165615X,3.0
4,276729,521795028,6.0
5,276733,2080674722,0.0
6,276736,3257224281,8.0
7,276737,600570967,6.0
8,276744,038550120X,7.0
9,276745,342310538,10.0


In [19]:
sample_data.to_csv("train_ratings.csv", index=False)

In [20]:
len(sample_data.movieid.unique())
# total unique movies/items
# sample_data = whole data

322729

In [21]:
# converting unique movies numpy ndarray to list
unique_movies = sample_data.movieid.unique()
type (list(unique_movies))
print(unique_movies)

['034545104X' 155061224 446520802 ... '048623715X' 486256588 515069434]


 ***ASSIGN* CONTINUOUS INDEX TO movieid**

In [22]:
# adding index to movie id thru a dict
original_movie_ids = []
mapped_movie_ids = []
movie_dict = {}
count = 1
for movie_id in unique_movies:
    original_movie_ids.append(movie_id)
    mapped_movie_ids.append(count)
    movie_dict[movie_id] = count
    count += 1

In [23]:
print(len(original_movie_ids))
print(len(mapped_movie_ids))

322729
322729


In [24]:
# creating a dataframe with colomn1 as uniquemovieids and column2 as mappedmovieids(aka index) , dataframe index vs movie index
movies_map_df_dict = {"original_movie_ids": original_movie_ids,
                      "mapped_movie_ids":mapped_movie_ids}
movies_map_df = pd.DataFrame(movies_map_df_dict)
print(movies_map_df.head())
movies_map_df.to_csv("mapped_movie_ids.csv",index=False)

  original_movie_ids  mapped_movie_ids
0         034545104X                 1
1          155061224                 2
2          446520802                 3
3         052165615X                 4
4          521795028                 5


In [185]:
len(sample_data)

200000

In [26]:
# adding a column(mapped_movie_id) to orginal wholedataset dataframe sample_data
# len(sample_data)
for i in range(len(sample_data)): # len(sample_data)
  mapped_movie_id = movie_dict[sample_data.iloc[i][1]] # int()
#     print(sample_data.iloc[i][1] ,mapped_movie_id )
  sample_data.loc[sample_data.index[i], 'mapped_movie_id'] = (mapped_movie_id) # int()
    

In [160]:
sample_data.head()

,userid,movieid,movieRating,mapped_movie_id
0,276725,034545104X,0.0,1.0
1,276726,155061224,5.0,2.0
2,276727,446520802,0.0,3.0
3,276729,052165615X,3.0,4.0
4,276729,521795028,6.0,5.0


In [199]:
sample_data = sample_data[:100000]

In [200]:
users = np.array(sample_data['userid'])
items = np.array(sample_data['movieid'])
ratings = np.array(sample_data['movieRating'])
mapped_movie_ids = np.array(sample_data['mapped_movie_id'])

In [201]:
print(users[3],items[3], ratings[3])

276729 052165615X 3.0


In [202]:
#sample_data.movieid.max() alpha numeric

In [203]:
len(users), len(items), len(ratings), len(mapped_movie_ids)

(100000, 100000, 100000, 100000)

In [204]:
len(sample_data.userid.unique())
# unique users in wholedataset

9424

In [205]:
len(sample_data.movieid.unique())
# unique movies in wholedataset

63797

In [206]:
# sample_data.to_sparse(fill_value=0)

Create Utility Matrix

In [221]:
users = users.astype(np.uint32)
ratings = ratings.astype(np.uint32)
mapped_movie_ids = mapped_movie_ids.astype(np.uint32)

In [222]:
print(len(users))

100000


In [223]:
# converting original dataframe sample_data to a matrix representation/utility matrix with the help of csr_matrix func of scipy
utility_csr = csr_matrix((ratings.astype(int), (users , mapped_movie_ids.astype(int))))

In [224]:
utility_csr.shape

(23289, 63798)

In [225]:
utility_matrix = csr_matrix((ratings.astype(int), (users, mapped_movie_ids.astype(int)))).toarray() # Users x Items 
utility_matrix_t = utility_matrix.T
utility_matrix

MemoryError: Unable to allocate 5.54 GiB for an array with shape (23289, 63798) and data type int32

In [90]:
#print(utility_matrix[users[4]][items[4]])

In [219]:
# we have an extra expendable row(likely first row), column in both user_utility_matrix and item_utility_matrix
# shud be 943x1682 / 1682x943 , somewhere csr_matrix func is adding an extra row
print(utility_matrix.shape)
print(utility_matrix_t.shape)

(23289, 63798)
(63798, 23289)


In [220]:
np.savetxt('users_m.txt', utility_matrix, fmt="%d") # All rows as a USERS
np.savetxt('items_m.txt', utility_matrix_t, fmt="%d") # All rows as a ITEMS

OSError: [Errno 28] No space left on device

In [ ]:
# # passing the above two txt files into java code for generating recommendation thru compressed bit vectors

In [ ]:
# below four cells revist

In [ ]:
mapped_movie_ids_dict = {}
for key in movie_dict:
    value = movie_dict[key]
    mapped_movie_ids_dict[value] = key

In [ ]:
def get_original_movie_ids(mapped_movie_ids):
    original_ids = []
    for movie_id in mapped_movie_ids:
        if int(movie_id) in mapped_movie_ids_dict:
            original_ids.append(mapped_movie_ids_dict[int(movie_id)])
        else:
            original_ids.append(-1)
    return original_ids

In [ ]:
temp_str = "6,15,18,19,24,25,26,27,31,32,33,231,291,321,462,470,471,473,477,620"

In [ ]:
bsi_mapped_movie_ids = temp_str.split(",")
bsi_original_movie_ids = get_original_movie_ids(bsi_mapped_movie_ids)
print(bsi_original_movie_ids)

[474, 29, 274, 1042, 118, 1, 546, 95, 246, 98, 193, 520, 558, 97, 870, 44, 686, 729, 566, 372]


In [ ]:
# now we evaluate compressed bit vector recommendations ( java code txt files)

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.sparse import *
from scipy import *
from sklearn.preprocessing import LabelEncoder

In [ ]:
test_data = pd.read_csv('test_ratings.csv')
nRow, nCol = test_data.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 15909 rows and 3 columns


In [ ]:
mapped_movieID_df = pd.read_csv('mapped_movie_ids.csv')

In [ ]:
test_data

,userid,movieid,movieRating
0,1,154,5
1,1,161,4
2,1,18,4
3,1,171,5
4,1,246,5
...,...,...,...
15904,943,399,1
15905,943,1330,3
15906,943,227,1
15907,943,722,3


In [ ]:
mapped_movieID_df.head()

,original_movie_ids,mapped_movie_ids
0,242,1
1,302,2
2,377,3
3,51,4
4,346,5


In [ ]:
# create dictionaries for movieID mapping
mapped_movieID_dict = {}
# mapped_movieID_df.size
for i in range(len(mapped_movieID_df)):
    row = mapped_movieID_df.loc[i]
#     print(row)
    mapped_movieID_dict[row.original_movie_ids] = row.mapped_movie_ids

In [ ]:
original_movieID_dict = {}
for i in range(len(mapped_movieID_df)):
    row = mapped_movieID_df.loc[i]
#     print(row)
    original_movieID_dict[row.mapped_movie_ids] = row.original_movie_ids

In [ ]:
# unique users in test data for evaluation
test_users = test_data.userid.unique()
test_users_list = test_users.tolist()
print(test_users_list)
print(len(test_users_list))
# used in the java file to generate test_recommendation_bsi.txt and bsi_recommendations_for_all_user.txt

[1, 2, 3, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 18, 21, 22, 23, 24, 25, 26, 28, 37, 38, 41, 42, 43, 44, 48, 49, 52, 54, 56, 57, 58, 59, 60, 62, 63, 64, 65, 69, 70, 72, 73, 75, 76, 77, 79, 81, 82, 83, 84, 85, 87, 89, 90, 91, 92, 94, 95, 96, 97, 99, 100, 101, 102, 104, 106, 109, 110, 113, 115, 116, 117, 118, 119, 121, 122, 123, 125, 128, 130, 135, 138, 141, 144, 145, 148, 151, 152, 154, 157, 158, 159, 160, 161, 164, 167, 168, 174, 176, 177, 178, 180, 181, 183, 184, 186, 187, 188, 189, 190, 193, 194, 195, 197, 198, 200, 201, 206, 207, 210, 213, 214, 215, 216, 217, 218, 221, 222, 223, 224, 226, 227, 230, 232, 233, 234, 235, 236, 239, 243, 244, 246, 248, 249, 250, 251, 253, 254, 255, 256, 257, 262, 263, 264, 267, 268, 269, 270, 271, 272, 274, 275, 276, 277, 279, 280, 283, 286, 287, 288, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 301, 303, 305, 307, 308, 311, 312, 313, 314, 315, 316, 318, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 334, 336, 338, 339, 342, 343, 

In [ ]:
def get_mapped_movieids(original_movieIds):
    mapped_movieIds= []
    for _id in original_movieIds:
        mapped_movieIds.append(mapped_movieID_dict[_id])
    return mapped_movieIds

In [ ]:
# get Get movies which is rated (>3.0)/(change depending on dataset) by user in test data for BSI
def get_test_rec_movieIds(userId):
    user_df = test_data[test_data.userid == userId]
    user_movie_df = user_df[user_df.movieRating > 5.0]
    original_movieIds = user_movie_df.movieid.unique()
    mapped_movieIds = get_mapped_movieids(original_movieIds)
    return mapped_movieIds


In [ ]:
def find_common(original_ids, rec_ids):
    original_ids_set = set(original_ids)
    common_count = 0
    if rec_ids[-1] == '':
        rec_ids.pop()
    for _id in rec_ids:
        if int(_id) in original_ids_set:
            common_count += 1
    return common_count

In [ ]:
mapped_movieID_df.head()

,original_movie_ids,mapped_movie_ids
0,242,1
1,302,2
2,377,3
3,51,4
4,346,5


In [ ]:
def getOrginalMoviesIds(mapped_movie_ids):
    original_movieIds= []
    for _id in mapped_movie_ids:
#         print(_id)
        original_movieIds.append(original_movieID_dict[int(_id)])
    return original_movieIds


In [ ]:
print(getOrginalMoviesIds(['3','4']))

[377, 51]


In [ ]:
### Calculate Precision and Recall

def getPrecision(total_count, common_count):
    if(total_count ==0):
        return 0
    return (common_count/total_count)

def getRecall(total_count, common_count):
    if(total_count ==0):
        return 1
    return (common_count/total_count)

def getf1score(precision, recall):
    if(precision == 0 and recall == 0 ):
        return 0
    return ((2 * precision * recall )/ ( precision + recall ))

In [ ]:
### Evalute Bsi algorithm for all test users


In [ ]:
# get_test_rec_movieIds
count = 0
input_file = open('test_recommendation_bsi.txt','r')

userIds = []
actual_movies_watched = []
recomendations = []
common_movies = []
similar_usres_k = []
precision = []
recall = []
f1score = []

while (True):
    line = input_file.readline()
    if not line:
        break
    count +=1
    
    values = line.split(", ")
    userid = values[0]
    userK = values[1]
    no_recomendations = values[2]
    recomendations_for_user = values[3:]
    
    recomendations_for_user.pop() ## removing last null(\n) value
    
    actual_movies_wated_by_users = get_test_rec_movieIds(int(userid)) # movies in test data  #
    common_count = find_common(actual_movies_wated_by_users, recomendations_for_user) # movies in algo's predictions
    
#     print(userId,userK,no_recomendations)
#     print(common_count)

    userIds.append(userid)
    similar_usres_k.append(int(userK))
    actual_movies_watched.append(len(actual_movies_wated_by_users)) 
    recomendations.append(int(no_recomendations))
    common_movies.append(common_count)
    
    total_count_for_precision = int(no_recomendations) # actual results
    total_count_for_recall = len(actual_movies_wated_by_users)
    
    temp_Precision = getPrecision(total_count_for_precision, common_count)
    precision.append(temp_Precision)
    temp_recall = getRecall(total_count_for_recall, common_count)
    recall.append(temp_recall)
    f1score.append(getf1score(temp_Precision, temp_recall))
    
print(count)
# 568(test_users) * userK(4 len of list) * itemK(5 len of list) = 11360 test_recommendation_bsi.txt rows 

11360


In [ ]:
# storing precision and recall in new dict and dataframe
result_df_dict = {"userId":userIds,"actual_movies_watched":actual_movies_watched,
                  "recomendations":recomendations,"common_movies":common_movies,
                  "similar_usres_k":similar_usres_k, "precision": precision, "recall": recall, "f1_score": f1score}

In [ ]:
result_bsi_df = pd.DataFrame(result_df_dict)
result_bsi_df.head(25)
result_bsi_df = result_bsi_df.astype({"userId": int})
result_bsi_df

,userId,actual_movies_watched,recomendations,common_movies,similar_usres_k,precision,recall,f1_score
0,1,28,5,0,5,0.00,0.000000,0.000000
1,1,28,10,0,5,0.00,0.000000,0.000000
2,1,28,25,1,5,0.04,0.035714,0.037736
3,1,28,50,1,5,0.02,0.035714,0.025641
4,1,28,100,2,5,0.02,0.071429,0.031250
...,...,...,...,...,...,...,...,...
11355,943,19,5,0,25,0.00,0.000000,0.000000
11356,943,19,10,0,25,0.00,0.000000,0.000000
11357,943,19,25,0,25,0.00,0.000000,0.000000
11358,943,19,50,1,25,0.02,0.052632,0.028986


In [ ]:
result_bsi_df.sort_values('precision', ascending=False)

,userId,actual_movies_watched,recomendations,common_movies,similar_usres_k,precision,recall,f1_score
9770,806,11,5,3,20,0.6,0.272727,0.375000
4670,379,28,5,3,20,0.6,0.107143,0.181818
4835,389,30,5,2,25,0.4,0.066667,0.114286
4235,339,39,5,2,25,0.4,0.051282,0.090909
4830,389,30,5,2,20,0.4,0.066667,0.114286
...,...,...,...,...,...,...,...,...
5876,465,8,10,0,25,0.0,0.000000,0.000000
5875,465,8,5,0,25,0.0,0.000000,0.000000
5874,465,8,100,0,20,0.0,0.000000,0.000000
5873,465,8,50,0,20,0.0,0.000000,0.000000


In [ ]:
result_bsi_df.sort_values('recall', ascending=False)

,userId,actual_movies_watched,recomendations,common_movies,similar_usres_k,precision,recall,f1_score
3294,275,3,100,2,20,0.02,0.666667,0.038835
3293,275,3,50,2,20,0.04,0.666667,0.075472
3289,275,3,100,2,10,0.02,0.666667,0.038835
3288,275,3,50,2,10,0.04,0.666667,0.075472
3284,275,3,100,2,5,0.02,0.666667,0.038835
...,...,...,...,...,...,...,...,...
6181,489,12,10,0,5,0.00,0.000000,0.000000
6180,489,12,5,0,5,0.00,0.000000,0.000000
1970,167,8,5,0,20,0.00,0.000000,0.000000
1971,167,8,10,0,20,0.00,0.000000,0.000000


In [ ]:
result_bsi_df.sort_values('f1_score', ascending=False)

,userId,actual_movies_watched,recomendations,common_movies,similar_usres_k,precision,recall,f1_score
9770,806,11,5,3,20,0.6,0.272727,0.375000
9776,806,11,10,3,25,0.3,0.272727,0.285714
9771,806,11,10,3,20,0.3,0.272727,0.285714
9766,806,11,10,3,10,0.3,0.272727,0.285714
1920,161,3,5,1,5,0.2,0.333333,0.250000
...,...,...,...,...,...,...,...,...
5641,452,22,10,0,5,0.0,0.000000,0.000000
5640,452,22,5,0,5,0.0,0.000000,0.000000
5639,451,7,100,0,25,0.0,0.000000,0.000000
5638,451,7,50,0,25,0.0,0.000000,0.000000


In [ ]:
result_bsi_df_5 = result_bsi_df.loc[result_bsi_df['recomendations']==5]
result_bsi_df_10 = result_bsi_df.loc[result_bsi_df['recomendations']==10]
result_bsi_df_25 = result_bsi_df.loc[result_bsi_df['recomendations']==25]
result_bsi_df_50 = result_bsi_df.loc[result_bsi_df['recomendations']==50]
result_bsi_df_100 = result_bsi_df.loc[result_bsi_df['recomendations']==100]

In [ ]:
result_bsi_df_5.mean()

userId                   461.948944
actual_movies_watched     15.360915
recomendations             5.000000
common_movies              0.156250
similar_usres_k           15.000000
precision                  0.031250
recall                     0.010309
f1_score                   0.014389
dtype: float64

In [ ]:
result_bsi_df_5.max()

userId                   943.000000
actual_movies_watched     70.000000
recomendations             5.000000
common_movies              3.000000
similar_usres_k           25.000000
precision                  0.600000
recall                     0.333333
f1_score                   0.375000
dtype: float64

In [ ]:
result_bsi_df_10.mean()

userId                   461.948944
actual_movies_watched     15.360915
recomendations            10.000000
common_movies              0.275088
similar_usres_k           15.000000
precision                  0.027509
recall                     0.018074
f1_score                   0.019811
dtype: float64

In [ ]:
result_bsi_df_10.max()

userId                   943.000000
actual_movies_watched     70.000000
recomendations            10.000000
common_movies              3.000000
similar_usres_k           25.000000
precision                  0.300000
recall                     0.333333
f1_score                   0.285714
dtype: float64

In [ ]:
result_bsi_df_25.mean()

userId                   461.948944
actual_movies_watched     15.360915
recomendations            25.000000
common_movies              0.543134
similar_usres_k           15.000000
precision                  0.021725
recall                     0.035589
f1_score                   0.024581
dtype: float64

In [ ]:
result_bsi_df_25.max()

userId                   943.000000
actual_movies_watched     70.000000
recomendations            25.000000
common_movies              4.000000
similar_usres_k           25.000000
precision                  0.160000
recall                     0.500000
f1_score                   0.195122
dtype: float64

In [ ]:
result_bsi_df_50.mean()

userId                   461.948944
actual_movies_watched     15.360915
recomendations            50.000000
common_movies              0.857394
similar_usres_k           15.000000
precision                  0.017148
recall                     0.055974
f1_score                   0.024437
dtype: float64

In [ ]:
result_bsi_df_100.mean()

userId                   461.948944
actual_movies_watched     15.360915
recomendations           100.000000
common_movies              1.296215
similar_usres_k           15.000000
precision                  0.012962
recall                     0.084777
f1_score                   0.021478
dtype: float64

In [ ]:
###

In [ ]:
userId = []
recommendations = []
input_file_1 = open('bsi_recommendations_for_all_users.txt','r')
while (True):
    line = input_file_1.readline()
    if not line:
        break
    count +=1
    values = line.split(", ")
    user_id = values[0]
    recomendations_for_user = values[1:]
    recomendations_for_user.pop()
    actual_movie_ids = getOrginalMoviesIds(recomendations_for_user)
    userId.append(user_id)
    recommendations.append(actual_movie_ids)
    
print("Finished")

Finished


In [ ]:
result_all_user_df_dict = {"userID": userId, "recommendations": recommendations}

In [ ]:
result_all_user_df = pd.DataFrame(result_all_user_df_dict)

In [ ]:
result_all_user_df # one user is missing 685/686 rows are 942

,userID,recommendations
0,1,"[474, 603, 4, 181, 196, 143, 423, 288, 427, 23..."
1,2,"[302, 144, 95, 246, 4, 181, 196, 248, 275, 174..."
2,3,"[242, 302, 346, 222, 1, 100, 515, 288, 690, 24..."
3,4,"[302, 222, 144, 1, 98, 332, 100, 181, 196, 237..."
4,5,"[474, 265, 144, 234, 98, 193, 603, 4, 196, 143..."
...,...,...
937,939,"[265, 144, 1, 234, 98, 332, 100, 181, 196, 143..."
938,940,"[265, 1, 234, 98, 193, 603, 241, 4, 100, 181, ..."
939,941,"[222, 144, 95, 98, 100, 196, 423, 288, 427, 24..."
940,942,"[194, 603, 100, 181, 196, 423, 515, 526, 427, ..."


# Collaborative Filtering pyspark ( ALS Baseline)

In [ ]:
!pip install pyspark

In [ ]:
import os
import numpy as np
from pyspark.sql import SparkSession
from pyspark import SparkContext

from pyspark.mllib.recommendation import ALS
import math
import pyspark.sql
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.ml.evaluation import RegressionEvaluator
import time


In [ ]:
# Calling spark session to register application
spark = SparkSession \
    .builder \
    .appName("Recom") \
    .config("spark.recom.demo", "1") \
    .getOrCreate()
# lambda word: (word, 1)

In [ ]:
ratings_df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("train_ratings.csv")

In [ ]:
ratings_df.show()
ratings_df.count()

+------+-------+-----------+
|userid|movieid|movieRating|
+------+-------+-----------+
|   196|    242|          3|
|   186|    302|          3|
|    22|    377|          1|
|   244|     51|          2|
|   166|    346|          1|
|   298|    474|          4|
|   115|    265|          0|
|   253|    465|          5|
|   305|    451|          0|
|     6|     86|          3|
|    62|    257|          2|
|   286|   1014|          5|
|   200|    222|          0|
|   210|     40|          3|
|   224|     29|          3|
|   303|    785|          3|
|   122|    387|          5|
|   194|    274|          0|
|   291|   1042|          4|
|   234|   1184|          2|
+------+-------+-----------+
only showing top 20 rows



100000

In [ ]:
ratings_df = ratings_df.drop('') # reduntant> no values are dropped
ratings_df.count()

100000

### Drop test data from training set

In [ ]:
ratings_df = ratings_df[ratings_df.movieRating != 0.0]
ratings_df.summary
ratings_df.count()

84091

In [ ]:
(trainingData,validationData,testData) = ratings_df.randomSplit([0.6,0.2,0.2],5) # randomSplit(weights, seed)

In [ ]:
trainingData.show()
print(trainingData.count())
print(validationData.count())
print(testData.count())

+------+-------+-----------+
|userid|movieid|movieRating|
+------+-------+-----------+
|     1|      2|          3|
|     1|      3|          4|
|     1|      6|          5|
|     1|      7|          4|
|     1|      9|          5|
|     1|     10|          3|
|     1|     12|          5|
|     1|     13|          5|
|     1|     14|          5|
|     1|     15|          5|
|     1|     16|          5|
|     1|     21|          1|
|     1|     22|          4|
|     1|     24|          3|
|     1|     25|          4|
|     1|     27|          2|
|     1|     28|          4|
|     1|     29|          1|
|     1|     31|          3|
|     1|     32|          5|
+------+-------+-----------+
only showing top 20 rows

50373
16870
16848


In [ ]:
validation_for_predict = validationData.select('userid','movieid')
test_for_predict = testData.select('userid','movieid')

In [ ]:
seed = 5 
iterations = 10
regularization_parameter = 0.1 
ranks = [4, 8, 12] 

In [ ]:
min_error = 1000
for rank in ranks:
    model = ALS.train(ratings_df, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    
    #converting prediction into key value pair like key=(userId,movieId) and value = rating
    predictions = model.predictAll(validation_for_predict.rdd).map(lambda r: ((r[0], r[1]), r[2]))
    
    #joining predicted rating and original ratings to calculate error
    rates_and_preds = validationData.rdd.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean()) 

    print ('For rank',rank, "the RMSE is ", error)
    if error < min_error:
        min_error = error
        best_rank = rank

print ("The best model was trained with rank", best_rank)

For rank 4 the RMSE is  0.8254553665866379
For rank 8 the RMSE is  0.7712129798364723
For rank 12 the RMSE is  0.7339375461866569
The best model was trained with rank 12


In [ ]:
predictions_test = model.predictAll(test_for_predict.rdd).map(lambda r: ((r[0], r[1]), r[2]))

In [ ]:
predictions_test.take(5)

[((436, 454), 3.7781829764828756),
 ((634, 1084), 3.032140186085046),
 ((526, 1084), 4.4657644308826265),
 ((506, 772), 2.1279597335641802),
 ((76, 772), 2.9846891068203982)]

### Get Recomendations For User

This method returns original movieId where CBV algorithm returns mapped movieId

In [ ]:
def getRecommendations(user,ratings_df,trainDf,model, k):
    
    userDf = ratings_df.filter(ratings_df.userid == user) # for eg user = 1 ...getting all movies(rows) user1 has rated 

    mov = ratings_df.select('movieid').subtract(userDf.select('movieid')) # mov dataframe wont have whatever user1 has rated
    
    pred_rat = model.predictAll(mov.rdd.map(lambda x: (user, x[0]))).collect()
    
    recommendations = sorted(pred_rat, key=lambda x: x[2], reverse=True)[:k]
    
    return recommendations

In [ ]:
model.predict(1,1084) # make sure product id is in the model

4.222420911402509

In [ ]:
user = 1
# how many recommendations you want
k= 10

# Call getRecommendations method
derived_rec = getRecommendations(user, ratings_df, trainingData, model, k)

print ("Movies recommended for:",user)
movie_ids = []
for i in range(len(derived_rec)):
    movie_ids.append(derived_rec[i][1])
#     print (i+1,derived_rec[i][1])

print(movie_ids)
#     movies_df.filter(movies_df.movieId==derived_rec[i][1]).select('title').show()

Movies recommended for: 1
[1512, 1449, 408, 1495, 390, 611, 1467, 1463, 511, 657]


In [ ]:
temp_str = "70, 235, 316, 333, 423, 441, 457, 480, 552, 553, 590, 4006, 72378, 538, 36, 300, 344, 380, 531, 371"

In [ ]:
bsi_movie_ids = temp_str.split(", ") # not using bsi_movie_ids
print(bsi_movie_ids)
print(movie_ids)

['70', '235', '316', '333', '423', '441', '457', '480', '552', '553', '590', '4006', '72378', '538', '36', '300', '344', '380', '531', '371']
[1512, 1449, 408, 1495, 390, 611, 1467, 1463, 511, 657]


In [ ]:
# test_users = [  1, 2, 3, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 18, 21,
#                 22, 23, 24, 25, 26, 28, 37, 38, 41, 42, 43, 44, 48, 49,
#                 52, 54, 56, 57, 58, 59, 60, 62, 63, 64, 65, 69, 70, 72,
#                 73, 75, 76, 77, 79, 81, 82, 83, 84, 85, 87, 89, 90, 91,
#                 92, 94, 95, 96, 97, 99, 100, 101, 102, 104, 106, 109, 110,
#                 113, 115, 116, 117, 118, 119, 121, 122, 123, 125, 128, 130,
#                 135, 138, 141, 144, 145, 148, 151, 152, 154, 157, 158, 159,
#                 160, 161, 164, 167, 168, 174, 176, 177, 178, 180, 181, 183,
#                 184, 186, 187, 188, 189, 190, 193, 194, 195, 197, 198, 200, 201,
#                 206, 207, 210, 213, 214, 215, 216, 217, 218, 221, 222, 223, 224, 226, 227,
#                 230, 232, 233, 234, 235, 236, 239, 243, 244, 246, 248, 249, 250, 251, 253,
#                 254, 255, 256, 257, 262, 263, 264, 267, 268, 269, 270, 271, 272, 274, 275,
#                 276, 277, 279, 280, 283, 286, 287, 288, 290, 291, 292, 293, 294, 295, 296,
#                 297, 298, 299, 301, 303, 305, 307, 308, 311, 312, 313, 314, 315, 316, 318,
#                 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 334, 336, 338,
#                 339, 342, 343, 344, 345, 346, 347, 348, 350, 354, 357, 360, 361, 363, 365, 367, 370,
#                 371, 372, 373, 374, 378, 379, 380, 381, 382, 383, 385, 387, 388, 389, 391, 392, 393, 394,
#                 395, 396, 397, 398, 399, 401, 402, 403, 405, 406, 407, 409, 411, 412, 413, 416, 417, 421, 422,
#                 423, 425, 426, 428, 429, 430, 432, 435, 436, 437, 440, 442, 445, 447, 449, 450, 451, 452, 453, 454,
#                 455, 456, 457, 458, 459, 460, 463, 464, 465, 466, 468, 470, 472, 474, 476, 478, 479, 480, 481, 483, 484,
#                 486, 487, 488, 489, 490, 492, 493, 495, 496, 497, 498, 499, 500, 501, 503, 504, 505, 506,
#                 507, 508, 514, 518, 521, 523, 524, 525, 526, 527, 528, 532, 533, 534, 535, 536, 537, 538,
#                 539, 540, 541, 542, 543, 545, 546, 548, 551, 552, 553, 554, 555, 557, 559, 560, 561, 562,
#                 566, 567, 568, 569, 573, 577, 579, 582, 585, 586, 587, 588, 591, 592, 593, 595, 600, 601,
#                 605, 606, 608, 610, 615, 617, 618, 619, 620, 621, 622, 624, 625, 627, 629, 630, 632, 633,
#                 634, 637, 638, 639, 640, 642, 643, 645, 647, 648, 650, 653, 654, 655, 658, 659, 660, 661,
#                 663, 664, 665, 666, 669, 671, 676, 679, 682, 683, 684, 686, 690, 693, 694, 697, 698, 699,
#                 704, 705, 707, 708, 709, 710, 711, 712, 715, 716, 717, 719, 721, 724, 727, 731, 733, 734,
#                 735, 738, 741, 745, 746, 747, 748, 749, 751, 752, 753, 756, 757, 758, 761, 763, 764, 766, 768,
#                 770, 771, 773, 774, 776, 778, 780, 782, 786, 787, 788, 790, 793, 795, 796, 798, 802, 804, 805,
#                 806, 807, 815, 821, 823, 825, 826, 828, 829, 830, 831, 833, 834, 835, 836, 838, 839, 840, 843,
#                 844, 846, 847, 848, 850, 851, 854, 860, 862, 863, 864, 865, 867, 868, 870, 871, 872, 875, 877,
#                 878, 880, 881, 882, 883, 885, 886, 887, 889, 890, 892, 893, 894, 896, 897, 899, 901, 903, 907, 908,
#                 910, 911, 912, 913, 916, 918, 919, 921, 922, 923, 924, 927, 930, 931, 932, 933, 934, 936, 938, 940, 942, 943]

test_users_list = [1, 2, 3, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 18, 21, 22, 23, 24, 25, 26, 28, 37, 38, 41, 42, 43, 44, 48, 49, 52, 54, 56, 57, 58, 59, 60, 62, 63, 64, 65, 69, 70, 72, 73, 75, 76, 77, 79, 81, 82, 83, 84, 85, 87, 89, 90, 91, 92, 94, 95, 96, 97, 99, 100, 101, 102, 104, 106, 109, 110, 113, 115, 116, 117, 118, 119, 121, 122, 123, 125, 128, 130, 135, 138, 141, 144, 145, 148, 151, 152, 154, 157, 158, 159, 160, 161, 164, 167, 168, 174, 176, 177, 178, 180, 181, 183, 184, 186, 187, 188, 189, 190, 193, 194, 195, 197, 198, 200, 201, 206, 207, 210, 213, 214, 215, 216, 217, 218, 221, 222, 223, 224, 226, 227, 230, 232, 233, 234, 235, 236, 239, 243, 244, 246, 248, 249, 250, 251, 253, 254, 255, 256, 257, 262, 263, 264, 267, 268, 269, 270, 271, 272, 274, 275, 276, 277, 279, 280, 283, 286, 287, 288, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 301, 303, 305, 307, 308, 311, 312, 313, 314, 315, 316, 318, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 334, 336, 338, 339, 342, 343, 344, 345, 346, 347, 348, 350, 354, 357, 360, 361, 363, 365, 367, 370, 371, 372, 373, 374, 378, 379, 380, 381, 382, 383, 385, 387, 388, 389, 391, 392, 393, 394, 395, 396, 397, 398, 399, 401, 402, 403, 405, 406, 407, 409, 411, 412, 413, 416, 417, 421, 422, 423, 425, 426, 428, 429, 430, 432, 435, 436, 437, 440, 442, 445, 447, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 463, 464, 465, 466, 468, 470, 472, 474, 476, 478, 479, 480, 481, 483, 484, 486, 487, 488, 489, 490, 492, 493, 495, 496, 497, 498, 499, 500, 501, 503, 504, 505, 506, 507, 508, 514, 518, 521, 523, 524, 525, 526, 527, 528, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 545, 546, 548, 551, 552, 553, 554, 555, 557, 559, 560, 561, 562, 566, 567, 568, 569, 573, 577, 579, 582, 585, 586, 587, 588, 591, 592, 593, 595, 600, 601, 605, 606, 608, 610, 615, 617, 618, 619, 620, 621, 622, 624, 625, 627, 629, 630, 632, 633, 634, 637, 638, 639, 640, 642, 643, 645, 647, 648, 650, 653, 654, 655, 658, 659, 660, 661, 663, 664, 665, 666, 669, 671, 676, 679, 682, 683, 684, 686, 690, 693, 694, 697, 698, 699, 704, 705, 707, 708, 709, 710, 711, 712, 715, 716, 717, 719, 721, 724, 727, 731, 733, 734, 735, 738, 741, 745, 746, 747, 748, 749, 751, 752, 753, 756, 757, 758, 761, 763, 764, 766, 768, 770, 771, 773, 774, 776, 778, 780, 782, 786, 787, 788, 790, 793, 795, 796, 798, 802, 804, 805, 806, 807, 815, 821, 823, 825, 826, 828, 829, 830, 831, 833, 834, 835, 836, 838, 839, 840, 843, 844, 846, 847, 848, 850, 851, 854, 860, 862, 863, 864, 865, 867, 868, 870, 871, 872, 875, 877, 878, 880, 881, 882, 883, 885, 886, 887, 889, 890, 892, 893, 894, 896, 897, 899, 901, 903, 907, 908, 910, 911, 912, 913, 916, 918, 919, 921, 922, 923, 924, 927, 930, 931, 932, 933, 934, 936, 938, 940, 942, 943]
print(test_users_list)
#test_users = test_data.userid.unique()
# userKs = [5,10,20,25] # add this in final run
# itemKs = [5, 10, 25, 50 ,100]
userKs = [5]
itemKs = [10]
len(test_users_list)

# 568(test_users) * userK(4 len of list) * itemK(5 len of list) = 11360 test_recommendation_bsi.txt rows 

[1, 2, 3, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 18, 21, 22, 23, 24, 25, 26, 28, 37, 38, 41, 42, 43, 44, 48, 49, 52, 54, 56, 57, 58, 59, 60, 62, 63, 64, 65, 69, 70, 72, 73, 75, 76, 77, 79, 81, 82, 83, 84, 85, 87, 89, 90, 91, 92, 94, 95, 96, 97, 99, 100, 101, 102, 104, 106, 109, 110, 113, 115, 116, 117, 118, 119, 121, 122, 123, 125, 128, 130, 135, 138, 141, 144, 145, 148, 151, 152, 154, 157, 158, 159, 160, 161, 164, 167, 168, 174, 176, 177, 178, 180, 181, 183, 184, 186, 187, 188, 189, 190, 193, 194, 195, 197, 198, 200, 201, 206, 207, 210, 213, 214, 215, 216, 217, 218, 221, 222, 223, 224, 226, 227, 230, 232, 233, 234, 235, 236, 239, 243, 244, 246, 248, 249, 250, 251, 253, 254, 255, 256, 257, 262, 263, 264, 267, 268, 269, 270, 271, 272, 274, 275, 276, 277, 279, 280, 283, 286, 287, 288, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 301, 303, 305, 307, 308, 311, 312, 313, 314, 315, 316, 318, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 334, 336, 338, 339, 342, 343, 

568

In [ ]:
# remaining_test_users = [487,488,489,490,491,492,493,495,
#               497,500,503,504,509,510,511,512,517,520,522,523,524,525,526,527,528,532,534,541,542,543,546,551,552,
#               554,555,558,559,560,561,562,563,564,566,567,570,571,572,573,577,579,582,583,584,585,586,587,588,590,
#               591,592,593,594,596,597,599,600,601,603,604,605,606,607,608] # this was prev dataset
# remaining_test_users not being used anywhere

In [ ]:
def recomendMoviesForAllUsers(test_users, itemKs,testData,trainingData,model):
    output_file = open("test_recommendations_als_5_10.txt", "a")
    for u in range(len(test_users)):
        user = test_users[u]
        for itemk in itemKs:
            derived_rec = getRecommendations(user,testData,trainingData,model,itemk)
            
            output_file.write(str(user)+", ")
            output_file.write(str(5)+", ")
            for i in range(5):
                output_file.write(str(derived_rec[i][1])+", ")
            output_file.write("\n")
                
            output_file.write(str(user)+", ")
            output_file.write(str(itemk)+", ")
            for i in range(len(derived_rec)):
                output_file.write(str(derived_rec[i][1])+", ")
            output_file.write("\n")

In [ ]:
start = time.time()
derived_rec = recomendMoviesForAllUsers(test_users_list,itemKs,testData,trainingData,model)
end = time.time()
print(end - start)


635.5835509300232


In [ ]:
###  Get movies which is rated (>3.0) by user in test data for ALS

def get_test_rec_movieIds_als(userId):
    user_df = test_data[test_data.userid == userId]
    user_movie_df = user_df[user_df.movieRating > 5.0]
    original_movieIds = user_movie_df.movieid.unique()
    return list(original_movieIds)

In [ ]:
# get_test_rec_movieIds
count = 0
input_file = open('test_recommendations_als_5_10.txt','r')

userIds = []
actual_movies_watched = []
recomendations = []
common_movies = []
precision = []
recall = []
f1score = []

while (True):
    line = input_file.readline()
    if not line:
        break
    count +=1
    values = line.split(", ")
    userId = values[0]
    no_recomendations = values[1]
    recomendations_for_user = values[2:]
    
    recomendations_for_user.pop() ## removing last null(\n) value
    
    actual_movies_watched_by_users = get_test_rec_movieIds_als(int(userId)) # movies in test data
#     print(count)
    common_count = find_common(actual_movies_watched_by_users, recomendations_for_user) # movies in algo's predictions
    
#     print(userId,userK,no_recomendations)
#     print(common_count)

    userIds.append(userId)
    actual_movies_watched.append(len(actual_movies_watched_by_users)) 
    recomendations.append(int(no_recomendations))
    common_movies.append(common_count)
    
    total_count_for_precision = int(no_recomendations)
    total_count_for_recall = len(actual_movies_watched_by_users)
    
    temp_Precision = getPrecision(total_count_for_precision, common_count)
    precision.append(temp_Precision)
    temp_recall = getRecall(total_count_for_recall, common_count)
    recall.append(temp_recall)
    f1score.append(getf1score(temp_Precision, temp_recall))
    
print(count)
# test_recommendations_als_5_10.txt row length

1136


In [ ]:
result_df_dict = {"userId":userIds,"actual_movies_watched":actual_movies_watched,
                  "recomendations":recomendations,"common_movies":common_movies,
                   "precision": precision, "recall": recall, "f1_score": f1score}

In [ ]:
result_als_df = pd.DataFrame(result_df_dict)

In [ ]:
result_als_df

,userId,actual_movies_watched,recomendations,common_movies,precision,recall,f1_score
0,1,28,5,0,0.0,0.000000,0.000000
1,1,28,10,0,0.0,0.000000,0.000000
2,2,8,5,0,0.0,0.000000,0.000000
3,2,8,10,0,0.0,0.000000,0.000000
4,3,1,5,0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...
1131,940,13,10,0,0.0,0.000000,0.000000
1132,942,12,5,0,0.0,0.000000,0.000000
1133,942,12,10,2,0.2,0.166667,0.181818
1134,943,19,5,1,0.2,0.052632,0.083333


In [ ]:
result_als_df = result_als_df.astype({"userId": int})


In [ ]:
result_als_df_5 = result_als_df.loc[result_als_df['recomendations']==5]
result_als_df_10 = result_als_df.loc[result_als_df['recomendations']==10]

In [ ]:
result_als_df_5.mean()

userId                   461.948944
actual_movies_watched     15.360915
recomendations             5.000000
common_movies              0.228873
precision                  0.045775
recall                     0.017520
f1_score                   0.023445
dtype: float64

In [ ]:
result_als_df_10.mean()

userId                   461.948944
actual_movies_watched     15.360915
recomendations            10.000000
common_movies              0.498239
precision                  0.049824
recall                     0.036173
f1_score                   0.038259
dtype: float64